# Reproduce Figure 1
- Py kernel with R script
- Verify by looking at [Figure 1 in the Original Analysis paper](https://www.nature.com/articles/s41598-021-87029-w?proof=t%25C2%25A0) and by running `emas.R` [Original version](https://github.com/usc-sail/tiles-day-night/blob/main/code/ground_truth/mgt_lm.R). Be sure to configure your file paths.

In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
base = "/Users/brinkley97/Documents/development/lab-kcad/"
path_to_file = "datasets/tiles_dataset/table_1/"
name_of_file = "mgt_lm.csv.gz"
mgt_lm_file = base + path_to_file + name_of_file

In [3]:
def load_data(file):
    
    original_data = pd.read_csv(file)
    copy_of_data = original_data.copy()
    
    return copy_of_data

In [4]:
mgt_df = load_data(mgt_lm_file)

# Load Generated Specific Questions

In [5]:
%run "../generateSpecificQuestions.ipynb"

In [6]:
figure_1_ema_sqs

['on both *work* day and *off* day, what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *EMA Anxiety*?',
 'on both *work* day and *off* day, what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *EMA Stess*?',
 'on both *work* day and *off* day, what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *EMA Positive Affect*?',
 'on both *work* day and *off* day, what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *EMA Negative Affect*?']

In [7]:
ema_values = ontology_mappings["ema"]
# ema_values

# Integrate R

In [8]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages

# load and activate() bc if NOT error (Conversion 'py2rpy' not defined for objects of type '<class 'pandas.core.frame.DataFrame'>') will appear
from rpy2.robjects import pandas2ri
pandas2ri.activate()

report = rpackages.importr('report')

# Produce Figure 1

In [9]:
def calculate_ema(ema_df, specific_ema):
    '''Integrate py and r to compare differences in reported EMAs (Anxiety, Stress, PA, and NA)
    
    Arguments:
    ema_df -- pd DataFrame (with all ema variables)
    specific_ema -- py str (of either Anxiety, Stress, PA, and NA)
    
    Return:
    linear mixed model (lmer) for each ema variable
    '''

    r_objects = robjects.r
    r_objects.source("emas.R")
    if specific_ema == 'EMA Anxiety':
        anxiety_df = r_objects.anxiety_model(ema_df)
        # print(anxiety_df)
        return anxiety_df
    elif specific_ema == 'EMA Stess':
        stress_df = r_objects.stress_model(ema_df)
        # print(stress_df)
        return stress_df
    elif specific_ema == 'EMA Positive Affect':
        pa_df = r_objects.pa_model(ema_df)
        # print(pa_df)
        return pa_df
    elif specific_ema == 'EMA Negative Affect':
        na_df = r_objects.na_model(ema_df)
        # print(na_df)
        return na_df
        

In [10]:
def figure_1(figure_1_specific_questions, sleep_df, ema_values):
    '''Reproduce Figure_1
    
    Arguments:
    figure_1_specific_questions -- py list
    sleep_df -- pd Dataframe
    
    Functions:
    calculate_anova()
    
    Return:
    nothing; everything is being printed in calculate_anova()
    '''
    
    for figure_1_specific_question_idx in range(len(figure_1_specific_questions)):
        figure_1_specific_question = figure_1_specific_questions[figure_1_specific_question_idx]
        print(figure_1_specific_question_idx, "figure_1_specific_question : ", figure_1_specific_question)
        
        word_in_specific_question = figure_1_specific_question.split("*")
        for ema_value in ema_values:
            
            if ema_value in word_in_specific_question:
                print(ema_value, True)
                calculate_ema(sleep_df, ema_value)
                print()
                print("============================================================================")
                print()
        

In [11]:
figure_1(figure_1_ema_sqs, mgt_df, ema_values)

R[write to console]: Loading required package: Matrix



0 figure_1_specific_question :  on both *work* day and *off* day, what are differences in primarily *day-shift* nurses and primarily *night-shift* nurses for *EMA Anxiety*?
EMA Anxiety True


R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 5947' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 5947)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 5947' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 5947)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: NOTE: Results may be misleading due to involvement in interactions



Linear mixed model fit by REML ['lmerMod']
Formula: anxiety ~ work + shift + shift:work + (1 | id)
   Data: df

REML criterion at convergence: 12104.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.8758 -0.5022 -0.1583  0.4115  6.0287 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept) 0.2026   0.4501  
 Residual             0.4214   0.6492  
Number of obs: 5947, groups:  id, 107

Fixed effects:
                    Estimate Std. Error t value
(Intercept)          1.44712    0.05770  25.079
workwork             0.13404    0.02190   6.121
shiftnight           0.06139    0.09337   0.657
workwork:shiftnight  0.04833    0.03632   1.331

Correlation of Fixed Effects:
            (Intr) wrkwrk shftng
workwork    -0.202              
shiftnight  -0.618  0.125       
wrkwrk:shft  0.122 -0.603 -0.207
 work emmean     SE  df asymp.LCL asymp.UCL
 off    1.48 0.0467 Inf      1.39      1.57
 work   1.64 0.0465 Inf      1.54      1.73

Results are averaged

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 5947' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 5947)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 5947' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 5947)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: NOTE: Results may be misleading due to involvement in interactions



Linear mixed model fit by REML ['lmerMod']
Formula: stressd ~ work * shift + (1 | id)
   Data: df

REML criterion at convergence: 14606.1

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.9065 -0.6259 -0.1919  0.5852  4.6215 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept) 0.2667   0.5164  
 Residual             0.6437   0.8023  
Number of obs: 5947, groups:  id, 107

Fixed effects:
                    Estimate Std. Error t value
(Intercept)          1.69525    0.06661  25.451
workwork             0.33916    0.02705  12.536
shiftnight           0.06183    0.10781   0.574
workwork:shiftnight -0.02936    0.04488  -0.654

Correlation of Fixed Effects:
            (Intr) wrkwrk shftng
workwork    -0.216              
shiftnight  -0.618  0.134       
wrkwrk:shft  0.130 -0.603 -0.221
 work emmean     SE  df asymp.LCL asymp.UCL
 off    1.73 0.0539 Inf      1.62      1.83
 work   2.05 0.0536 Inf      1.95      2.16

Results are averaged over the lev

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 5947' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 5947)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 5947' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 5947)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: NOTE: Results may be misleading due to involvement in interactions



Linear mixed model fit by REML ['lmerMod']
Formula: pand_PosAffect ~ work * shift + (1 | id)
   Data: df

REML criterion at convergence: 31808

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.6242 -0.5694 -0.0652  0.5748  3.9047 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept) 18.71    4.326   
 Residual             11.36    3.371   
Number of obs: 5947, groups:  id, 107

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         13.17807    0.53904  24.447
workwork             0.24617    0.11381   2.163
shiftnight          -0.07439    0.87124  -0.085
workwork:shiftnight -0.30020    0.18876  -1.590

Correlation of Fixed Effects:
            (Intr) wrkwrk shftng
workwork    -0.113              
shiftnight  -0.619  0.070       
wrkwrk:shft  0.068 -0.603 -0.115
 work emmean    SE  df asymp.LCL asymp.UCL
 off    13.1 0.436 Inf      12.3      14.0
 work   13.2 0.435 Inf      12.4      14.1

Results are averaged over the l

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'pbkrtest.limit = 5947' (or larger)
[or, globally, 'set emm_options(pbkrtest.limit = 5947)' or larger];
but be warned that this may result in large computation time and memory use.

R[write to console]: Note: D.f. calculations have been disabled because the number of observations exceeds 3000.
To enable adjustments, add the argument 'lmerTest.limit = 5947' (or larger)
[or, globally, 'set emm_options(lmerTest.limit = 5947)' or larger];
but be warned that this may result in large computation time and memory use.



Linear mixed model fit by REML ['lmerMod']
Formula: pand_NegAffect ~ work * shift + (1 | id)
   Data: df

REML criterion at convergence: 25635.2

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.2411 -0.4248 -0.1638  0.1723  7.6379 

Random effects:
 Groups   Name        Variance Std.Dev.
 id       (Intercept) 3.253    1.804   
 Residual             4.072    2.018   
Number of obs: 5947, groups:  id, 107

Fixed effects:
                    Estimate Std. Error t value
(Intercept)          6.36538    0.22762  27.966
workwork             0.07368    0.06810   1.082
shiftnight          -0.05165    0.36808  -0.140
workwork:shiftnight  0.25967    0.11296   2.299

Correlation of Fixed Effects:
            (Intr) wrkwrk shftng
workwork    -0.159              
shiftnight  -0.618  0.099       
wrkwrk:shft  0.096 -0.603 -0.163
$emmeans
 work shift emmean    SE  df asymp.LCL asymp.UCL
 off  day     6.37 0.228 Inf      5.92      6.81
 work day     6.44 0.227 Inf      5.99      6.88
 of